In [3]:
import pandas as pd
import numpy as np

from datetime import date

### Importando dataset e-commerce

Para mas informacion del dataset, consulte en el siguiente [link](https://www.kaggle.com/datasets/carrie1/ecommerce-data) 

In [4]:
ecommerce = pd.read_csv("data.csv")

In [5]:
ecommerce.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [6]:
ecommerce.shape

(541909, 8)

### cast de fechas y tratamiento de columnas Dias, mes, año

In [7]:
ecommerce.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity       float64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [8]:
ecommerce["InvoiceDate"] = pd.to_datetime(ecommerce["InvoiceDate"])

In [9]:
ecommerce.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [10]:
ecommerce.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom


In [11]:
ecommerce["year"] = ecommerce["InvoiceDate"].dt.year.astype('Int64')
ecommerce["month"] = ecommerce["InvoiceDate"].dt.month.astype('Int64')
ecommerce["day"] = ecommerce["InvoiceDate"].dt.day.astype('Int64')

In [12]:
ecommerce.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,1


#### Exploracion de paises

In [13]:
ecommerce["Country"].drop_duplicates()

0               United Kingdom
26                      France
109                        NaN
197                  Australia
385                Netherlands
1109                   Germany
1236                    Norway
1404                      EIRE
5320               Switzerland
6421                     Spain
6608                    Poland
7134                  Portugal
7214                     Italy
7279                   Belgium
7986                 Lithuania
9783                     Japan
14938                  Iceland
20000          Channel Islands
20017                  Denmark
29732                   Cyprus
30078                   Sweden
31464                  Austria
31982                   Israel
34083                  Finland
38313                  Bahrain
69007                   Greece
69623                Hong Kong
70758                Singapore
72985                  Lebanon
89570     United Arab Emirates
100810            Saudi Arabia
103598          Czech Republic
119191  

### Ejercicios

**Ejercicio 1:** Por pais muestre cual fue el año en el que mejor ingreso se registro y su valor, adicional muestre cual fue el año con mas Invoices (Frecuencia) y otra columna con el total.

Nota: Note que el año con mejor ingreso no es lo mismo con el año con mas invoices, por ejemplo en 2019 Honduras en dos invoices se facturo 10,000. sin embargo, en 2020 se realizaron 20 invoices (pueda que en este año el ingreso de esas invoices hayan sumado 7,000 inferior al 2020) 

*recuerde que el ejemplo es inventado*

Ejemplo de como se deberia de ver el dataframe resultante:


In [14]:
pd.DataFrame({
    "country" : ["Honduras"]
    , "YearMejorIngreso" : [2019]
    , "ValorYearMejorIngreso" : [20000]
    , "YearMejorFrecuenciaInvoices": [2020]
    , "CantidadYearMejorFrecuenciaInvoices": [20]
})

,country,YearMejorIngreso,ValorYearMejorIngreso,YearMejorFrecuenciaInvoices,CantidadYearMejorFrecuenciaInvoices
0,Honduras,2019,20000,2020,20


*Answer:*

In [15]:
dt = ecommerce.sort_values(["Country","year","UnitPrice","Quantity","InvoiceNo"]);
dt["TotalInvoice"] = dt["Quantity"] * dt["UnitPrice"]

mejorIngreso = dt.groupby(["Country", "year"], as_index = False)["TotalInvoice"].sum()
temp = mejorIngreso.groupby(["Country"], as_index = False)["TotalInvoice"].max()
mejorIngreso = mejorIngreso.merge(temp, on=["Country", "TotalInvoice"], how="inner")

mejorInvoices = dt.groupby(["Country", "year"], as_index =  False)["InvoiceNo"].count()
temp2 = mejorInvoices.groupby(["Country"], as_index = False)["InvoiceNo"].max()
mejorInvoices = mejorInvoices.merge(temp2, on=["Country", "InvoiceNo"], how="inner")

resultado = mejorIngreso.merge(mejorInvoices, on=["Country"], how="inner")
resultado = resultado.rename(columns = {"year_x": "YearMejorIngreso", "TotalInvoice":"ValorYearMejorIngreso", "year_y":"YearMejorFrecuenciaInvoices", "InvoiceNo": "CantidadYearMejorFrecuenciaInvoices"})
resultado

,Country,YearMejorIngreso,ValorYearMejorIngreso,YearMejorFrecuenciaInvoices,CantidadYearMejorFrecuenciaInvoices
0,Australia,2011,134288.610,2011,1216
1,Austria,2011,9876.940,2011,390
2,Bahrain,2011,342.660,2011,18
3,Belgium,2011,38919.070,2011,1965
4,Brazil,2011,1143.600,2011,32
5,Canada,2011,3599.680,2011,147
6,Channel Islands,2011,19587.010,2011,736
7,Cyprus,2011,11271.290,2011,534
8,Czech Republic,2011,707.720,2011,30
9,Denmark,2011,17383.680,2011,365


**Ejercicio 2:** Cual es el Top tres por año de los Invoices de lo mas facturado

Ejemplo de como se deberia de ver el dataframe resultante (Recuerde que este es un ejemplo inventado):

In [38]:
pd.DataFrame({
    "Year" : [2019,2019,2019,2020,2020,2020]
    , "Top" : [1,2,3,1,2,3]
    , "InvoiceNo" : [536365, 536999, 693365, 24987, 56832, 254232]    
    , "TotalInvoice": [30000,28000,27900,29000,25000,24600]
})

,Year,Top,InvoiceNo,TotalInvoice
0,2019,1,536365,30000
1,2019,2,536999,28000
2,2019,3,693365,27900
3,2020,1,24987,29000
4,2020,2,56832,25000
5,2020,3,254232,24600


*Answer:*

In [210]:
montosPorFactura = ecommerce.sort_values(["InvoiceNo","year","UnitPrice"], ascending=True)
montosPorFactura["TotalInvoice"] = montosPorFactura["Quantity"]  * montosPorFactura["UnitPrice"]
montosPorFactura = montosPorFactura.groupby(["InvoiceNo","year"], as_index= False)["TotalInvoice"].sum()
montosPorFactura = montosPorFactura.sort_values(["TotalInvoice"], ascending=False)

montosPorFactura["Top"] = montosPorFactura.groupby(["year"], as_index= False).cumcount()
montosPorFactura["Top"] = montosPorFactura["Top"] + 1
montosPorFactura = montosPorFactura.loc[montosPorFactura["Top"] < 4, ["year","Top","InvoiceNo", "TotalInvoice"] ]
montosPorFactura

,year,Top,InvoiceNo,TotalInvoice
2301,2011,1,541431,77183.60
18757,2011,2,574941,52940.94
19447,2011,3,576365,50653.91
1644,2010,1,539750,18745.86
673,2010,2,537659,15885.49
651,2010,3,537632,13541.33


**Ejercicio 3:** Genere el siguiente Dataset para cada uno de los clientes:

In [ ]:
pd.DataFrame({
    "CustomerID" : [111,111,111]
    , "YearMonth" : ["2019-1","2019-2","2019-3"]
    , "TotalValorInvoices" : [20000,22000,0]    
    , "TotalCantidadInvoices" : [3,2,0]    
    , "FacturaSiguienteMes": [1,0,1]
})

,CustomerID,YearMonth,TotalValorInvoices,TotalCantidadInvoices,FacturaSiguienteMes
0,111,2019-1,20000,3,1
1,111,2019-2,22000,2,0
2,111,2019-3,0,0,1


*Ten en cuenta lo siguiente:*

Mediante el cross join puedo emular el producto cartesiano de dos vectores para obtener las combinaciones unicas de los valores de cada uno de ellos

In [ ]:
df1 = pd.DataFrame( {"year": [2019,2020]} )
df2 = pd.DataFrame( {"Cliente": [111,112,113]} )

df2.merge(df1, how='cross')

,Cliente,year
0,111,2019
1,111,2020
2,112,2019
3,112,2020
4,113,2019
5,113,2020


*Observaciones:*

* TotalValorInvoices: La suma del total facturado por todas sus facturas
* TotalCantidadInvoices: La cantidad de facturas unicas realizadas en ese mes
* FacturaSiguienteMes: Una variable de respuesta que nos dice si al siguiente mes por ese cliente se encontro por lo menos una factura.

Note que por la logica de este dataset el ultimo año-mes encontrado en todo el dataset se va a descartar del resultado final por que no se sabe el valor del siguiente mes. 1: si se encuentra una factura al siguiente mes, 0: no se encuentra nada.

*Answer:*

In [75]:
ej3 = ecommerce.sort_values(["CustomerID", "year", "month"], ascending = True)
ej3["TotalValorInvoices"] = ej3["Quantity"] * ej3["UnitPrice"]
ej3 = ej3.groupby(["CustomerID", "year", "month", "InvoiceDate"], as_index = False)["TotalValorInvoices"].sum()

TotalCantidadInvoices = ecommerce.groupby(["CustomerID", "year", "month" ], as_index=False)["InvoiceNo"].count()

ej4 = ej3.merge(TotalCantidadInvoices, on=["CustomerID", "year", "month"], how="inner")
ej4["YearMonth"] = ej4["year"].astype("string") + '-' + ej4["month"].astype("string")

ej4["row_number"] = ej4.groupby(["CustomerID", "year", "month"], as_index=False).cumcount()
ej4 = ej4.loc[ej4["row_number"] == 0, ["CustomerID","YearMonth","TotalValorInvoices","InvoiceNo"]]
ej4["FacturaSiguienteMes"] = ej4.groupby(["CustomerID"], as_index = False)["InvoiceNo"].shift(-1)

ej4 = ej4.loc[pd.isnull(ej4["FacturaSiguienteMes"]), ["CustomerID", "YearMonth", "TotalValorInvoices", "InvoiceNo", "FacturaSiguienteMes"]]
ej4 = ej4.rename(columns = {"InvoiceNo": "TotalCantidadInvoices"}).fillna(0)
ej4.head(10)

,CustomerID,YearMonth,TotalValorInvoices,TotalCantidadInvoices,FacturaSiguienteMes
0,12346.0,2011-1,77183.60,2,0.0
8,12347.0,2011-12,224.82,11,0.0
12,12348.0,2011-9,310.00,3,0.0
13,12349.0,2011-11,1757.55,73,0.0
14,12350.0,2011-2,334.40,17,0.0
25,12352.0,2011-11,311.73,15,0.0
26,12353.0,2011-5,89.00,4,0.0
27,12354.0,2011-4,1079.40,58,0.0
28,12355.0,2011-5,459.40,13,0.0
31,12356.0,2011-11,58.35,2,0.0


**Ejercicio cuatro:** Cual es el top 3 de los productos que menos ingresos generan por Pais del ultimo año encontrado del dataset.

Ejemplo de como se deberia de ver el dataframe resultante (Recuerde que este es un ejemplo inventado): 

In [ ]:
pd.DataFrame({
    "Country" : ["Canada","Canada","Canada","Spain","Spain","Spain"]
    , "Top" : [1,2,3,1,2,3]    
    , "StockCode": [546,6886,665,656,888,668]
    , "Description": ["Product X","Product K","Product J","Product C","Product D","Product U"]
    , "TotalOnInvoices": [2000,2100,4000,3000,3800,3900]
})

,Country,Top,StockCode,Description,TotalOnInvoices
0,Canada,1,546,Product X,2000
1,Canada,2,6886,Product K,2100
2,Canada,3,665,Product J,4000
3,Spain,1,656,Product C,3000
4,Spain,2,888,Product D,3800
5,Spain,3,668,Product U,3900


*Answer:*

In [176]:
ej4 = ecommerce
ej4["TotalOnInvoices"] = ej4["Quantity"] * ej4["UnitPrice"] 
ej4 = ej4.groupby(["year", "Country", "StockCode", "Description"], as_index = False)["TotalOnInvoices"].sum()
ej4 = ej4.sort_values(["Country", "year", "TotalOnInvoices"], ascending=True)

tempC = ej4.sort_values(["year", "Country"], ascending=False)
tempC = ej4.groupby(["Country"], as_index = False)["year"].max()

ej4 = ej4.merge(tempC, on = ["Country", "year"], how = "inner")
ej4["Top"] = ej4.groupby(["Country"], as_index = False).cumcount()
ej4["Top"] = ej4["Top"] + 1
ej4 = ej4.loc[ej4["Top"] < 4, ["Country","Top", "StockCode", "Description", "TotalOnInvoices"]]
ej4.head(50)


,Country,Top,StockCode,Description,TotalOnInvoices
0,Australia,1,22232,JIGSAW TOADSTOOLS 3 PIECE,0.00
1,Australia,2,22738,RIBBON REEL SNOWY VILLAGE,0.00
2,Australia,3,22990,COTTON APRON PANTRY DESIGN,0.00
592,Austria,1,21402,RED EGG SPOON,2.88
593,Austria,2,21403,BLUE EGG SPOON,2.88
594,Austria,3,22027,TEA PARTY BIRTHDAY CARD,5.04
891,Bahrain,1,72802B,OCEAN SCENT CANDLE IN JEWELLED BOX,-180.24
892,Bahrain,2,37446,MINI CAKE STAND WITH HANGING CAKES,11.60
893,Bahrain,3,22697,GREEN REGENCY TEACUP AND SAUCER,17.70
907,Belgium,1,22845,VINTAGE CREAM CAT FOOD CONTAINER,-19.05
